# `Aula 1`
# `Conhecendo a base de dados`

### 1.1. Importando os dados


In [ ]:
import pandas as pd

In [ ]:
database = 'arquives/aluguel.csv'

df = pd.read_csv(
    database,
    sep=';'
)

df

In [ ]:
df.head(10)

In [ ]:
df.tail(10)

### 1.2. Características gerais da base de dados

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
# Visualizando uma coluna específica
df['Tipo']

In [ ]:
df[['Bairro', 'Tipo', 'Valor', 'Area']]

# `Aula 2`
# `Análise exploratória de dados`

### 2.1. Qual o valor médio de aluguel por tipo de imóvel?

In [ ]:
# Separando apenas as colunas que irei utilizar
tipo_valor = df[['Tipo', 'Valor']]
tipo_valor

In [ ]:
# Agrupando valores por tipo e calculando a média por agrupamento
df.groupby('Tipo').mean(numeric_only=True)

# agrupando por uma coluna específica
df.groupby('Tipo')[['Valor']].mean()

In [ ]:
# Agrupando valores por tipo e calculando a soma por agrupamento
df.groupby('Tipo')[['Valor']].sum()

In [ ]:
# Agrupando valores por tipo e calculando o número de reptições por agrupamento
df.groupby('Tipo')[['Valor']].count()

In [ ]:
# Se quisermos com que a tabela seja mostrada em forma de DataFrame, basta colacar o mais um par de [] em ['Valor']
df.groupby('Tipo')[['Valor', 'Condominio', 'IPTU']].mean()

In [ ]:
# Podemos também ordenar esses valores com a função 'sort_values()'
df_valor_tipo = df.groupby('Tipo')[['Valor']].mean().sort_values('Valor')

In [ ]:
df_valor_tipo.plot(kind='barh', figsize=(14, 10), color='purple')

### 2.2. Removendo os imóveis comerciais

In [ ]:
# fazenod um 'set' de um Series do nosso DataFrame
df['Tipo'].unique()

In [ ]:
imoveis_comerciais = ['Conjunto Comercial/Sala', 
                      'Prédio Inteiro', 'Loja/Salão', 
                      'Galpão/Depósito/Armazém', 
                      'Casa Comercial', 'Terreno Padrão',
                      'Loja Shopping/ Ct Comercial',
                      'Box/Garagem', 'Chácara',
                      'Loteamento/Condomínio', 'Sítio',
                      'Pousada/Chalé', 'Hotel', 'Indústria']                                            

In [ ]:
# Criando um DataFrame sem os tipos de imóveis comerciais
imoveis_residenciais = df.query('@imoveis_comerciais not in Tipo')
imoveis_residenciais

In [ ]:
# Vando tipos únicos do novo DataFrame
imoveis_residenciais['Tipo'].unique()

In [ ]:
# Plotando gráfico
imoveis_residenciais_media = imoveis_residenciais.groupby('Tipo')[['Valor']].mean().sort_values('Valor')
imoveis_residenciais_media.plot(kind='barh', figsize=(14, 10), color='blue')

### 2.3. Qual o percentual de cada tipo de imóvel na nossa base de dados?

In [ ]:
# contando o número de vezes que um grupo repete
imoveis_residenciais[['Tipo']].value_counts().to_frame()

In [ ]:
# trazendo o valor de repetições de um determinado grupo em porcetangem, transformando para DataFrame e ordenando em ordem descendente
df_imoveis_residenciais = imoveis_residenciais['Tipo'].value_counts(normalize=True).to_frame().sort_values('proportion', ascending=True)
df_imoveis_residenciais.rename(columns={'proportion': 'Percentual'}, inplace=True)
df_imoveis_residenciais = df_imoveis_residenciais.reset_index()
df_imoveis_residenciais

In [ ]:
df_imoveis_residenciais.plot(
    kind='bar',
    x='Tipo',
    y='Percentual',
    rot=80,
    figsize=(10, 5)    
)


### 2.3. Selecionando apenas os imóveis do tipo apartamento

In [ ]:
apartamentos = df.query('Tipo == "Apartamento"')
apartamentos.head(10)

# `Aula 3` 
# `Tratando e filtrando os dados`

### 3.1. Lidando com dados nulos

In [ ]:
# dataframe de mesmo tamanho do original indicando quais campo são, ou não, nulos
null_values = apartamentos.isnull()
print(null_values)
print()

# Essa função retorna se há ou não valores nulos no dataframe
has_nulls = apartamentos.isnull().any().any()
print(has_nulls)
print()

# Contando quantos nulos há em cada coluna do dataframa
null_counts = apartamentos.isnull().sum()
print(null_counts)

In [ ]:
from IPython.display import display

# Display no dataframe antigo para comparação
display(apartamentos)

# Tratando valores nulos com interpolação linear
interpolate_apartamentos = apartamentos.interpolate(method='linear')
display(interpolate_apartamentos)
apartamentos.dtypes

In [ ]:
# # Verificando se ainda há valores nulos no dataframe
interpolate_apartamentos.isnull().sum()

In [ ]:
interpolate_apartamentos['Tipo'].unique()
interpolate_apartamentos

### 3.2. Removendo registros

In [ ]:
interpolate_apartamentos.drop('Tipo', axis=1, inplace=True)
interpolate_apartamentos

### 3.3. Filtros

3.3.1 Apartamentos que possuem **1 quarto** e **aluguel menor que 1200**



In [ ]:
filter_one = interpolate_apartamentos[
    (interpolate_apartamentos['Quartos'] == 1)
    &
    (interpolate_apartamentos['Valor'] < 1200)
]
print(filter_one.shape)
filter_one

3.3.2. **Apartamentos** que possuem pelo menos **2 quartos**, **aluguel menor que 3000** e **area maior que 70**

In [ ]:
filter_two = interpolate_apartamentos[
    (interpolate_apartamentos['Quartos'] >= 2)
    &
    (interpolate_apartamentos['Valor'] < 3000)
    &
    (interpolate_apartamentos['Area'] > 70)
]
print(filter_two.shape)
filter_two

### 3.4. Salvando os dados

In [ ]:
"""
DataFrame inicial com os devidos tratamentos feitos, como: 
· 
""" 
interpolate_apartamentos.to_csv(
    "arquives/dados_apartamentos.csv", 
    index=False,
    sep=';'
)

In [ ]:
# Salvando os dois filtros feitos em DataFrames diferentes
filter_one.to_csv(
    'arquives/filter_1.csv',
    index=False,
    sep=';'
)

filter_two.to_csv(
    'arquive/filter_2.csv',
    index=False,
    sep=';'
)

In [ ]:
pd.read_csv("arquives/dados_apartamentos.csv")

# `Aula 4`
# `Manipulando os dados`

### 4.1. Criando colunas numéricas

### 4.2. Criando colunas categóricas